In [5]:
def inputify(fileName):
    return fileName
def outputify(fileName):
    return fileName

In [6]:
%matplotlib widget

import pandas as pd
import json
import numpy as np
import csv
import os
import matplotlib
import matplotlib.pyplot as plt
import pickle
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.externals import joblib
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from time import localtime
from datetime import datetime as dt
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from pathlib import Path

working_directory     = str(os.getcwd()) + "/"
# working_directoryPT = str(Path.home()) + "/"
# wd                  = "/Users/Jonat/Dropbox/CreateDawg/ImRight/input/"

In [26]:
# latest player data 
latesetPlayerdataSolo = pd.read_csv('LCS_Data/latestPlayerSolo.csv')
latesetPlayerdataObj  = pd.read_csv('LCS_Data/latestPlayerObj.csv')

latesetPlayerdataSolo['playerUnd'] = latesetPlayerdataSolo['player'].map(lambda x: x.lower())
latesetPlayerdataObj['playerUnd']  = latesetPlayerdataObj['player'].map(lambda x: x.lower())

In [27]:
def prepData(rp,bp):
    teamRSolo = latesetPlayerdataSolo[latesetPlayerdataSolo['playerUnd']==rp.lower()].copy()
    teamBSolo = latesetPlayerdataSolo[latesetPlayerdataSolo['playerUnd']==bp.lower()].copy()
    teamRObj  = latesetPlayerdataObj[latesetPlayerdataObj['playerUnd']==rp.lower()].copy()
    teamBObj  = latesetPlayerdataObj[latesetPlayerdataObj['playerUnd']==bp.lower()].copy()
    teamRSolo.drop(['player'],axis=1,inplace=True)
    teamBSolo.drop(['player'],axis=1,inplace=True)
    teamBObj.drop(['player'],axis=1,inplace=True)
    teamRObj.drop(['player'],axis=1,inplace=True)
    
    teamRSolo.drop(['playerUnd'],axis=1,inplace=True)
    teamBSolo.drop(['playerUnd'],axis=1,inplace=True)
    teamBObj.drop(['playerUnd'],axis=1,inplace=True)
    teamRObj.drop(['playerUnd'],axis=1,inplace=True)
    lane      = np.append(teamRSolo,teamBSolo)
    
    
    return lane,teamRObj,teamBObj
def prepObj(rTop,bTop,rJng,bJng,rMid,bMid,rBot,bBot,rSup,bSup):
    teamRObj = np.add(rTop,np.add(rJng,np.add(rMid,np.add(rBot,rSup))))
    teamBObj = np.add(bTop,np.add(bJng,np.add(bMid,np.add(bBot,bSup))))
    
    obj      = np.append(teamRObj,teamBObj)
    return obj
    
    

In [43]:
top,rObjTop,bObjTop = prepData('Niles','Huni')
jng,rObjJng,bObjJng = prepData('Iconic','Spica')
mid,rObjMid,bObjMid = prepData('Ablazeolive','PowerOfEvil')
bot,rObjBot,bObjBot = prepData('Stixxay','Lost')
sup,rObjSup,bObjSup = prepData('Newbie','SwordArt')
obj                 = prepObj(rObjTop,bObjTop,rObjJng,bObjJng,rObjMid,bObjMid,rObjBot,bObjBot,rObjSup,bObjSup)

[ 1.06239320e-03  7.45150431e-09  4.06609622e+02  2.22676485e-01
  4.29394470e-01  2.59350458e-01  2.67805800e+02  9.54092259e+00
  4.70769954e+03  9.16358337e+01 -2.72915928e+02 -3.57842724e+01
  3.59886269e+00  7.84645198e+03  1.53631494e+02 -2.04442684e+02
  6.40323681e+01  8.61518375e+00  6.83096453e-01  1.68608624e-01
  6.23263122e-01  5.19507129e-02]


ValueError: Empty data passed with indices specified.

In [38]:
teamR = []
teamB = []
stackedTbl = []
positions = {"top":top,"jng":jng,"mid":mid,"bot":bot,"sup":sup}
gameDataObj = obj
modelObj    = joblib.load("objectiveModels_LCK/obj_2.pkl")
scalerObj   = joblib.load("scalers_LCK/sObj_2.pkl")
for pos in positions.keys():
        relevantD = positions[pos]
        relevantM = joblib.load("pvpModels_LCK/"+pos+"_2.pkl")
        relevantS = joblib.load("scalers_LCK/"+pos+"_scaler_2.pkl")
        
        scaledRD  = relevantS.transform([relevantD])
        pvpProbs  = relevantM.predict_proba(scaledRD)
        teamR.append(pvpProbs[0][0])
        teamB.append(pvpProbs[0][1])
scaledObjData = scalerObj.transform([gameDataObj])
objResult = modelObj.predict(scaledObjData)
teamR.append(objResult[0])
stackedTbl.append(teamR + teamB)

In [39]:
rl = []
bl = []
for pos in positions.keys():
    rl.append("r_"+pos+'_pred')
    bl.append("b_"+pos+'_pred')

stackedDF = pd.DataFrame(data=stackedTbl, columns = rl + ['obj_pred'] + bl)
stackedDF.obj_pred.replace(('Red', 'Blue'), (0, 1), inplace=True)
# stackedDF.drop(['obj_pred'],axis=1,inplace=True)
stackedDF

,r_top_pred,r_jng_pred,r_mid_pred,r_bot_pred,r_sup_pred,obj_pred,b_top_pred,b_jng_pred,b_mid_pred,b_bot_pred,b_sup_pred
0,0.562846,0.356648,0.582326,0.394978,0.651102,0,0.437154,0.643352,0.417674,0.605022,0.348898


In [40]:
relevantMFinal = joblib.load("finalModel_LCK/final_2.pkl")
relevantSFinal = joblib.load("scalers_LCK/final_2.pkl")
scaledRDFinal  = relevantSFinal.transform(stackedDF)
ProbsFinal     = relevantMFinal.predict_proba(scaledRDFinal)
Predict        = relevantMFinal.predict(scaledRDFinal)

In [41]:
print(ProbsFinal,Predict)

[[0.56979997 0.43020003]] ['Blue']
